In [1]:
%pip install h5py
%pip install typing-extensions
%pip install wheel
%pip install transformers nest-asyncio torch
%pip install aiogram==2.25.1
%pip install ollama
%pip install langchain
%pip install babel==2.10.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [15]:
from aiogram import Bot, Dispatcher, executor, types
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import AutoTokenizer, RobertaForCausalLM, RobertaConfig
import torch
import re
import nest_asyncio
from aiogram import Bot, Dispatcher, types
import torch
import ollama
import logging
from collections import defaultdict
import time
from cachetools import TTLCache
import requests
from aiogram import Bot, Dispatcher, types
from aiogram.contrib.middlewares.logging import LoggingMiddleware
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from aiogram.dispatcher import FSMContext
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.utils import executor
import requests

In [3]:
# !ollama pull llama3


In [ ]:
# !curl https://ollama.ai/install.sh | sh 
# /home/instinctlol/DL_project/install.sh - в терминале!

In [4]:
!curl -X POST http://localhost:11434/api/pull -d '{"model":"llama2-uncensored" }'

{"status":"pulling manifest"}
{"status":"pulling 6aa74acf170f","digest":"sha256:6aa74acf170f8fb8e6ff8dae9bc9ea918d3a14b6ba95d0b0287da31b09a4848c","total":3825807040,"completed":3825807040}
{"status":"pulling 8c17c2ebb0ea","digest":"sha256:8c17c2ebb0ea011be9981cc3922db8ca8fa61e828c5d3f44cb6ae342bf80460b","total":7020,"completed":7020}
{"status":"pulling 7c23fb36d801","digest":"sha256:7c23fb36d80141c4ab8cdbb61ee4790102ebd2bf7aeff414453177d4f2110e5d","total":4766,"completed":4766}
{"status":"pulling 28577ba2177f","digest":"sha256:28577ba2177f0dfae2d86751cf3b0df2344694c41e1c4bbee934f2486b6cffc9","total":55,"completed":55}
{"status":"pulling 0025f348941e","digest":"sha256:0025f348941e96cfb2ce7128ad299ec350731336a2530a05f9c97e51d84fe2f2","total":39,"completed":39}
{"status":"pulling c67e365e770d","digest":"sha256:c67e365e770dd0aac228e429f7b4e50c187319d8f4586bca757c68b55dc81fca","total":529,"completed":529}
{"status":"verifying sha256 digest"}
{"status":"writing manifest"}
{"status":"removing

In [ ]:
# !curl -X POST http://localhost:11434/api/pull -d '{"model":"llama3" }'

In [5]:
!ollama pull llama2-uncensored

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling 6aa74acf170f... 100% ▕████████████████▏ 3.8 GB                         
pulling 8c17c2ebb0ea... 100% ▕████████████████▏ 7.0 KB                         
pulling 7c23fb36d801... 100% ▕████████████████▏ 4.8 KB                         
pulling 28577ba2177f... 100% ▕████████████████▏   55 B                         
pulling 0025f348941e... 100% ▕████████████████▏   39 B                         
pulling c67e365e770d... 100% ▕████████████████▏  529 B                         
verifying sha256 digest ⠋ pulling manifest 
pulling 6aa74acf170f... 100% ▕████████████████▏ 3.8 GB                         
pulling 8c17c2ebb0ea... 100% ▕████████████████▏ 7.0 KB                         
pulling 7c23fb36d801... 100% ▕████████████████▏ 4.8 KB                         
pulling 28577ba2177f... 100%

In [6]:

response = ollama.chat(model='llama2-uncensored', messages=[
  {
    'role': 'user',
    'content': 'Who is Moai?',
  },
])
print(response['message']['content'])
# little check if the model works just fine

Moai was a monolithic human statue constructed by the ancient Rapa Nui people on Easter Island. They were carved from volcanic rock and erected between 1250 AD to 1600 AD as guardians of their civilization.


In [20]:
# import nest_asyncio
# from aiogram import Bot, Dispatcher, executor, types
# import ollama
# import logging

# # Инициализация бота
# API_TOKEN = '7135255841:AAFZ0pVLhf7YI_wq6qZdH-VFXQZWyD_R9ec'
# bot = Bot(token=API_TOKEN)
# dp = Dispatcher(bot)


# logging.basicConfig(level=logging.INFO)

# @dp.message_handler()
# async def handle_message(message: types.Message):
#     user_input = message.text 

#     try:
#         # Запрос к модели
#         response = ollama.chat(model='llama2-uncensored', messages=[ #llama3
#             {
#                 'role': 'user',
#                 'content': user_input,
#             },
#         ])
#         # if user_input == ''
#         quote = response['message']['content']
#     except Exception as e:
#         logging.error(f"Ошибка при запросе к модели: {e}")
#         quote = "Произошла ошибка при обработке вашего запроса. Пожалуйста, попробуйте позже."

#     # Отправка сгенерированного ответа
#     await message.reply(quote)

# if __name__ == '__main__':
#     nest_asyncio.apply()  # Разрешение вложенных циклов asyncio
#     executor.start_polling(dp, skip_updates=True)


In [1]:
%pip install cachetools
%pip install googletrans

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
# Инициализация бота
API_TOKEN = '7135255841:AAFZ0pVLhf7YI_wq6qZdH-VFXQZWyD_R9ec'
bot = Bot(token=API_TOKEN)
dp = Dispatcher(bot)

# Настройка логирования
logging.basicConfig(level=logging.INFO)

# Кэширование ответов
cache = TTLCache(maxsize=100, ttl=600)

# Ограничение количества запросов
user_requests = defaultdict(list)
REQUEST_LIMIT = 5
TIME_WINDOW = 60  # Время в секундах

# Список администраторов (исключения по количеству запросов)
ADMIN_USERS = [399989639]  # Замените на ваши ID

# Словарь для хранения истории сообщений
user_message_history = defaultdict(list)

@dp.message_handler(commands=['weather'])
async def weather(message: types.Message):
    await message.reply("Введи свой город, дорогой")
    await WeatherStates.waiting_for_city.set()

class WeatherStates(StatesGroup):
    waiting_for_city = State()

@dp.message_handler(commands=['weather'])
async def weather(message: types.Message):
    await message.reply("Введи свой город, дорогой")
    await WeatherStates.waiting_for_city.set()

@dp.message_handler(state=WeatherStates.waiting_for_city)
async def handle_message(message: types.Message, state: FSMContext):
    city = message.text.lower()
    await message.reply("Ваш город: " + city)

    # Формируем запрос
    url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&units=metric&lang=ru&appid=79d1ca96933b0328e1c7e3e7a26cb347'
    try:
            # Отправляем запрос на сервер и сразу получаем результат
            weather_data = requests.get(url).json()

            if weather_data.get('cod') != 200:
                await message.reply("Не удалось найти погоду для указанного города. Пожалуйста, попробуйте снова.")
            else:
                # Получаем данные о температуре и о том, как она ощущается
                temperature = round(weather_data['main']['temp'])
                temperature_feels = round(weather_data['main']['feels_like'])

                # Выводим значения на экран
                now = f'Сейчас в городе {city} {temperature} °C\n'
                feels_like = f'Ощущается как {temperature_feels} °C'
                await message.reply(now + feels_like)
    except Exception as e:
            await message.reply(f"Произошла ошибка: {e}")
    # Завершаем состояние
    await state.finish()

@dp.message_handler(commands=['help'])
async def send_help(message: types.Message):
    help_text = ("Я бот, который использует модель Llama для ответов на ваши вопросы. "
                 "Просто отправьте мне сообщение, и я постараюсь вам помочь!\n"
                 "Доступные команды:\n"
                 "/help - Показать это сообщение.")
    await message.reply(help_text)

@dp.message_handler()
async def handle_message(message: types.Message):
    user_input = message.text.lower()  # Приведение запроса к нижнему регистру
    user_id = message.from_user.id
    username = message.from_user.username
    current_time = time.time()

    logging.info(f"Received message from {username} (ID: {user_id}): {user_input}")

    # Проверка на администратора
    if user_id not in ADMIN_USERS:
        # Очистка старых запросов
        user_requests[user_id] = [req for req in user_requests[user_id] if current_time - req < TIME_WINDOW]

        if len(user_requests[user_id]) >= REQUEST_LIMIT:
            await message.reply("Вы превысили лимит запросов. Пожалуйста, подождите некоторое время перед отправкой нового запроса.")
            return

        user_requests[user_id].append(current_time)

    # Получаем историю сообщений пользователя
    user_history = user_message_history[user_id]

    # Добавляем новое сообщение пользователя в историю
    user_history.append({
        'role': 'user',
        'content': user_input,
    })

    # Ограничиваем историю последними N сообщениями
    MAX_HISTORY = 5  # Максимальное количество сообщений в истории
    user_history = user_history[-MAX_HISTORY:]

    # Проверка, есть ли ответ в кэше
    if user_input in cache:
        quote = cache[user_input]
    else:
        try:
            # Запрос к модели Llama с учетом истории
            response = ollama.chat(model='llama2-uncensored', messages=user_history)
            quote = response['message']['content']

            # Сохранение ответа в кэше
            cache[user_input] = quote
            user_message_history[user_id] = user_history
        except Exception as e:
            logging.error(f"Error while querying the model: {e}")
            quote = "Произошла ошибка при обработке вашего запроса. Пожалуйста, попробуйте позже."

    logging.info(f"Replying to {username} (ID: {user_id}): {quote}")
    await message.reply(quote)

if __name__ == '__main__':
    nest_asyncio.apply()  # Разрешение вложенных циклов asyncio
    executor.start_polling(dp, skip_updates=True)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 58)

In [27]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/facebookresearch/audiocraft to /tmp/pip-install-6ou2b0ko/audiocraft_70ce8bdd29d84000be1a7b0fd24386f5
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft /tmp/pip-install-6ou2b0ko/audiocraft_70ce8bdd29d84000be1a7b0fd24386f5
  Resolved https://github.com/facebookresearch/audiocraft to commit 72cb16f9fb239e9cf03f7bd997198c7d7a67a01c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 3.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 3.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [31]:
!python3 -m pip install -U audiocraft

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch

In [ ]:
model = musicgen.MusicGen.get_pretrained('medium', device='cuda')
model.set_generation_params(duration=8)

In [ ]:
res = model.generate([
    'crazy EDM, heavy bang', 
    'classic reggae track with an electronic guitar solo',
    'lofi slow bpm electro chill with organic samples',
    'rock with saturated guitars, a heavy bass line and crazy drum break and fills.',
    'earthy tones, environmentally conscious, ukulele-infused, harmonic, breezy, easygoing, organic instrumentation, gentle grooves',
], 
    progress=True)
display_audio(res, 32000)

In [ ]:
@dp.message_handler(commands=['music'])
async def send_help(message: types.Message):
    help_text = ("Я бот, который использует модель Llama для ответов на ваши вопросы. "
                 "Просто отправьте мне сообщение, и я постараюсь вам помочь!\n"
                 "Доступные команды:\n"
                 "/help - Показать это сообщение.")
    await message.reply(help_text)